# Assignment 1

<b>Group 82</b>
* <b> Student 1 </b> : Joris Willems, 0908753
* <b> Student 2 </b> : Lars Schilders, 0908729

**Reading material**
* [1] Mikolov, Tomas, et al. "[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)" Advances in neural information processing systems. 2013. 

<b><font color='red'>NOTE</font></b> When submitting your notebook, please make sure that the training history of your model is visible in the output. This means that you should **NOT** clean your output cells of the notebook. Make sure that your notebook runs without errors in linear order.



# Question 1 - Keras implementation (10 pt)

### Word embeddings
Build word embeddings with a Keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training. Use a window size of 2 to train the embeddings (`window_size` in the jupyter notebook). 

1. Build word embeddings of length 50, 150 and 300 using the Skipgram model
2. Build word embeddings of length 50, 150 and 300 using CBOW model
3. Analyze the different word embeddings:
    - Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in the paper. Do not use existing libraries for this task such as Gensim. 
Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. $e_{x}$ denotes the embedding of word $x$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.
    - Give at least 5 different  examples of analogies.
    - Compare the performance on the analogy tasks between the word embeddings and briefly discuss your results.

4. Discuss:
  - Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


<b>HINT</b> See practical 3.1 for some helpful code to start this assignment.


### Import libraries

In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [11]:
import numpy as np
import keras.backend as K
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence

# other helpful libraries
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
import pandas as pd

In [3]:
print(tf.__version__) #  check what version of TF is imported

2.1.0


### Import file

If you use Google Colab, you need to mount your Google Drive to the notebook when you want to use files that are located in your Google Drive. Paste the authorization code, from the new tab page that opens automatically when running the cell, in the cell below.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Navigate to the folder in which `alice.txt` is located. Make sure to start path with '/content/drive/My Drive/' if you want to load the file from your Google Drive.

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/2IMM10 - Deep Learning/'

/content/drive/My Drive/Colab Notebooks/2IMM10 - Deep Learning


In [9]:
file_name = 'alice.txt'
corpus = open(file_name).readlines()

### Data preprocessing

See Practical 3.1 for an explanation of the preprocessing steps done below.

In [10]:
# Removes sentences with fewer than 3 words
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# remove punctuation in text and fit tokenizer on entire corpus
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)

# convert text to sequence of integer values
corpus = tokenizer.texts_to_sequences(corpus)
n_samples = sum(len(s) for s in corpus) # total number of words in the corpus
V = len(tokenizer.word_index) + 1 # total number of unique words in the corpus

In [16]:
print("Number of words in corpus: {} \nNumber of words in vocabulary: {}".format(n_samples, V))

Number of words in corpus: 27165 
Number of words in vocabulary: 2557


In [17]:
# example of how word to integer mapping looks like in the tokenizer
print(list((tokenizer.word_index.items()))[:5])

[('the', 1), ('and', 2), ('to', 3), ('a', 4), ('it', 5)]


In [24]:
# parameters
window_size = 2
window_size_corpus = 4

## Task 1.1 - Skipgram
Build word embeddings of length 50, 150 and 300 using the Skipgram model.

In [12]:
#prepare data for skipgram
def generate_data_skipgram(corpus, window_size, V):

    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    # Add the input word
                    #in_words.append(word)
                    all_in.append(word)
                    # Add one-hot of the context words
                    all_out.append(to_categorical(words[i], V))
                                      
    return (np.array(all_in),np.array(all_out))

In [27]:
# create training data
x , y = generate_data_skipgram(corpus,window_size,V)

In [243]:
# create skipgram architecture
def build_skipgram(dim):
    skipgram = Sequential()
    skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
    skipgram.add(Reshape((dim, )))
    skipgram.add(Dense(V, kernel_initializer='glorot_uniform', activation='softmax'))
    skipgram.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.1))
    return skipgram

<b>HINT</b>: To increase training speed of your model, you can use the free available GPU power in Google Colab. Go to `Edit` --> `Notebook Settings` --> select `GPU` under `hardware accelerator`.

In [259]:
# train skipgram model
skipgram_50 = build_skipgram(50)
skipgram_50.fit(x, y, batch_size=64, epochs=50)
skipgram_150 = build_skipgram(150)
skipgram_150.fit(x, y, batch_size=64, epochs=50)
skipgram_300 = build_skipgram(300)
skipgram_300.fit(x, y, batch_size=64, epochs=50)

Train on 94556 samples
Epoch 1/50
94556/94556 [==============================] - 4s 43us/sample - loss: 7.2228
Epoch 2/50
94556/94556 [==============================] - 4s 39us/sample - loss: 6.5966
Epoch 3/50
94556/94556 [==============================] - 4s 41us/sample - loss: 6.3870
Epoch 4/50
94556/94556 [==============================] - 5s 52us/sample - loss: 6.2722
Epoch 5/50
94556/94556 [==============================] - 4s 41us/sample - loss: 6.1929
Epoch 6/50
94556/94556 [==============================] - 4s 43us/sample - loss: 6.1324
Epoch 7/50
94556/94556 [==============================] - 4s 44us/sample - loss: 6.0832
Epoch 8/50
94556/94556 [==============================] - 4s 42us/sample - loss: 6.0420
Epoch 9/50
94556/94556 [==============================] - 4s 43us/sample - loss: 6.0064
Epoch 10/50
94556/94556 [==============================] - 4s 41us/sample - loss: 5.9750
Epoch 11/50
94556/94556 [==============================] - 4s 43us/sample - loss: 5.9469
Epoch 1

In [260]:
# save embeddings for vectors of length 50, 150 and 300 using skipgram model
embedding_50 = skipgram_50.get_weights()[0]
embedding_150 = skipgram_150.get_weights()[0]
embedding_300 = skipgram_300.get_weights()[0]

In [263]:
np.save("embedding50.npy",  embedding_50)
np.save("embedding150.npy", embedding_150)
np.save("embedding300.npy", embedding_300)

## Task 1.2 - CBOW

Build word embeddings of length 50, 150 and 300 using CBOW model.

In [36]:
# prepare data for CBOW
def generate_data_cbow(corpus, window_size, V):
    
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            
            context = [words[i] for i in range(p, n) if ((i != index) and (0 <= i < L))]
            context = [(maxlen - len(context)) * [0] + context]
            
            all_in.append(context)

            all_out.append(to_categorical(word, V))
                                      
    return (np.array(all_in),np.array(all_out))

# create training data
x_cbow, y_cbow = generate_data_cbow(corpus,window_size, V)

In [36]:
# create CBOW architecture
def build_cbow(dim):
    cbow = Sequential()
    cbow.add(Embedding(input_dim=, output_dim=, embeddings_initializer='glorot_uniform', input_length=))
    cbow.add(Reshape(( , )))
    cbow.add(Dense( , kernel_initializer='glorot_uniform', activation='softmax'))
    cbow.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.1))
    return skipgram

In [36]:
# train CBOW model

In [36]:
# save embeddings for vectors of length 50, 150 and 300 using CBOW model

## Task 1.3 - Analogy function

Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in [1]. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. 

In a perfect scenario, we would like that this analogy ( $e_{king} - e_{queen} + e_{woman}$) results in the embedding of the word "man". However, it does not always result in exactly the same word embedding. The result of the formula is called the expected or the predicted word embedding. In this context, "man" is called the true or the actual word $t$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.  

You have to answer an analogy function using each embedding for both CBOW and Skipgram model. This means that for each analogy we have 6 outputs. Show the true word (with distance similarity value between predicted embedding and true word embedding, i.e. `sim1`) , the predicted word (with distance similarity value between predicted embedding and the embedding of the word in the vocabulary that is closest to this predicted embedding, i.e. `sim2`) and a boolean answer whether the predicted word **exactly** equals the true word. 

<b>HINT</b>: to visualize the results of the analogy tasks , you can print them in a table. An example is given below.


| Analogy task | True word (sim1)  | Predicted word (sim2) | Embedding | Correct?|
|------|------|------|------|------|
|  queen is to king as woman is to ?	 | man (sim1) | predictd_word(sim2) | SG_50 | True / False|

* Give at least 5 different  examples of analogies.
* Compare the performance on the analogy s between the word embeddings and briefly discuss your results.

In [41]:
skipgram_embedding_50 = np.load('embedding/embedding50.npy')
skipgram_embedding_150 = np.load('embedding/embedding150.npy')
skipgram_embedding_300 = np.load('embedding/embedding300.npy')

# cbow_embedding_50 = 
# cbow_embedding_150 = 
# cbow_embedding_300 = 

In [42]:
def embed(word, embedding, vocab_size = V, tokenizer=tokenizer):
    # get the index of the word from the tokenizer, i.e. convert the string to it's corresponding integer in the vocabulary
    int_word = tokenizer.texts_to_sequences([word])[0]
    # get the one-hot encoding of the word
    bin_word = to_categorical(int_word, V)
    return np.dot(bin_word, embedding)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

def findAnalogy(prediction, embedding):
    
    all_word_embeddings = embed(list(tokenizer.word_index.keys()), embedding)
    
    similarity = cosine_similarity(all_word_embeddings, prediction)

    word_idx = similarity.argsort(axis=0)[-2,0]

    return list(tokenizer.word_index.keys())[word_idx]



In [56]:
tokenizer.word_counts.items()

odict_items([('chapter', 12), ('i', 545), ('down', 102), ('the', 1640), ('rabbit', 51), ('hole', 5), ('alice', 395), ('was', 356), ('beginning', 14), ('to', 726), ('get', 46), ('very', 144), ('tired', 7), ('of', 512), ('sitting', 10), ('by', 58), ('her', 248), ('sister', 9), ('on', 190), ('bank', 3), ('and', 869), ('having', 10), ('nothing', 33), ('do', 81), ('once', 34), ('or', 77), ('twice', 5), ('she', 551), ('had', 176), ('peeped', 3), ('into', 67), ('book', 11), ('reading', 3), ('but', 170), ('it', 592), ('no', 90), ('pictures', 4), ('conversations', 1), ('in', 367), ('what', 140), ('is', 108), ('use', 18), ('a', 631), ('thought', 74), ('without', 26), ('so', 151), ('considering', 3), ('own', 10), ('mind', 11), ('as', 263), ('well', 62), ('could', 75), ('for', 152), ('hot', 7), ('day', 28), ('made', 29), ('feel', 8), ('sleepy', 5), ('stupid', 6), ('whether', 11), ('pleasure', 2), ('making', 8), ('daisy', 1), ('chain', 1), ('would', 83), ('be', 148), ('worth', 4), ('trouble', 6), (

In [47]:
emb = skipgram_embedding_50

prediction = embed("brother", emb) - embed("sister", emb) + embed("father", emb)

print(cosine_similarity(prediction, embed("mother", emb)))

findAnalogy(prediction, emb)

ValueError: Found array with 0 sample(s) (shape=(0, 50)) while a minimum of 1 is required by check_pairwise_arrays.

## Task 1.4 - Discussion
Answer the following question:
* Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


# Question 2 - Peer review (0 pt):
Finally, each group member must write a single paragraph outlining their opinion on the work distribution within the group. Did every group member
contribute equally? Did you split up tasks in a fair manner, or jointly worked through the exercises. Do you think that some members of your group deserve a different grade from others? You can use the table below to make an overview of how the tasks were divided:



| Student name | Task  |
|------|------|
|  student name 1  | task x |
| student name 2  | task x|
| everyone | task x|
